In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from nltk.corpus import stopwords
from collections import Counter
import string
import re
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torchtext
from torchtext.vocab import GloVe

c:\Users\Alireza\anaconda3\envs\torch-gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
def load_data(csv_path):

  def preprocess_string(s):
    # Remove all non-word characters (everything except numbers and letters)
    s = re.sub(r"[^\w\s]", '', s)
    # Replace all runs of whitespaces with no space
    s = re.sub(r"\s+", '', s)
    # replace digits with no space
    s = re.sub(r"\d", '', s)

    return s

  def padding_(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
      if len(review) != 0:
        features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

  # Reading data from disk
  df = pd.read_csv(csv_path)
  # Replacing string labels with numbers
  df['sentiment'].replace({'positive': 1, 'negative': 0}, inplace=True)

  # dividing the dataset into train, validation, and test portions
  X,y = df['review'].values,df['sentiment'].values
  x_train1, x_test1, y_train, y_test = train_test_split(X, y, train_size=0.85, random_state=1)
  x_train1, x_val1, y_train, y_val = train_test_split(x_train1, y_train, train_size=0.823, random_state=1)

  print(f'Number of training examples: {len(x_train1)}')
  print(f'Number of validation examples: {len(x_val1)}')
  print(f'Number of test examples: {len(x_test1)}')


  # Building the vocabulary
  glove = GloVe(name='6B', dim=100, max_vectors=25000, cache = "./vectors_cache")

  x_train, x_val, x_test = [],[],[]
  for sent in x_train1:
    x_train.append([glove.stoi[preprocess_string(word)] for word in sent.lower().split() if preprocess_string(word) in glove.stoi])

  for sent in x_val1:
    x_val.append([glove.stoi[preprocess_string(word)] for word in sent.lower().split() if preprocess_string(word) in glove.stoi])

  for sent in x_test1:
    x_test.append([glove.stoi[preprocess_string(word)] for word in sent.lower().split() if preprocess_string(word) in glove.stoi])

  # Padding the data
  x_train_pad = padding_(x_train,500)
  x_val_pad = padding_(x_val,500)
  x_test_pad = padding_(x_test,500)

  # creating Tensor Datasets and DataLoaders
  train_data = TensorDataset(torch.from_numpy(x_train_pad), torch.from_numpy(y_train))
  valid_data = TensorDataset(torch.from_numpy(x_val_pad), torch.from_numpy(y_val))
  test_data = TensorDataset(torch.from_numpy(x_test_pad), torch.from_numpy(y_test))

  batch_size = 100

  train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
  valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
  test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

  return train_loader, valid_loader, test_loader

In [5]:
trainloader, validloader, testloader = load_data("IMDB Dataset.csv")

Number of training examples: 34977
Number of validation examples: 7523
Number of test examples: 7500


./vectors_cache\glove.6B.zip: 862MB [02:50, 5.05MB/s]                               
100%|█████████▉| 24999/25000 [00:01<00:00, 21624.36it/s]
